In this Notebook, we will be creating an authoring script to run 'cifar_10_cnn.py' execution script, which is bascically a CNN model trained on MNIST dataset
The model will be trained in Azure ML studio. (We can also train locally)
Steps Performed in this NB:
    
    1. Connect to workspace via the config.json file. Create an Experiment named : 'CIFAR_10_online'.
    2. Connect to a Azure compute target. Either connect an existing compute cluster, or make a new one.
    3. Create an RunConfig file, which will contain all the information about the environment parameters that the compute cluster requires to run 'cifar_10_cnn.py' successfully
    4. Create a Script Run Config object, which has the execution environment information
    5. Submit the run to the experiment env. and wait for completion

In [ ]:
#Importing required dependencies
#Workspace, Experiment, Run related packages
from azureml.core import Workspace, Experiment, RunConfiguration, ScriptRunConfig

#Packages for setting up online compute clusters
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#We will run the execution script using conda dependencies on the cluster
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

import os

In [ ]:
#Setting up workspace and experiment
ws = Workspace.from_config(path='/home/aditya/.azureml/config.json')
exp = Experiment(workspace=ws, name = 'CIFAR_10_online')

In [ ]:
#Setting up compute targets
#First let us check what compute cluster configurations are available
AmlCompute.supported_vmsizes(workspace=ws)

In [ ]:
#For this task, we will choose Standard_D2_v2
cluster_name = 'cluster-cifar'
min_nodes = 0
max_nodes = 4
vm_size = 'Standard_D2_v2'

#Now the next few statements will: Load the cluster if it already exists, or create a new cluster
try:
    aml_cluster = ComputeTarget(workspace=ws, name = cluster_name)
except ComputeTargetException:
    #This will be raised if the given cluster_name doesn't exists
    config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                   min_nodes = min_nodes,
                                                   max_nodes = max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name = cluster_name,
                                      provisioning_configuration=config)

#Now we will wait for completion
aml_cluster.wait_for_completion(show_output = True)


In [ ]:
#Now creating Run Config
run_amlcompute = RunConfiguration()

#Specifying compute target
run_amlcompute.target = aml_cluster

#Enabling docker
run_amlcompute.environment.docker.enabled = True
run_amlcompute.environment.docker.base_image = DEFAULT_CPU_IMAGE

#Specifying python packages to be used in the script
run_amlcompute.environment.python.user_managed_dependencies = False
run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(pip_packages = [
                                                        'azureml-sdk[notebooks, contrib, automl, explain]',
                                                        'azureml-tensorboard',
                                                        'tensorflow==2.1.0'
                                                                 ])

In [ ]:
#Specifying the training script for execution and the folder it is located in
script = 'cifar_10_cnn.py'
script_folder = os.getcwd()

#creating scriptrunconfig object
src = ScriptRunConfig(source_directory = script_folder,
                      script = script,
                      run_config = run_amlcompute)

#Creating run
run = exp.submit(src)

#Waiting for run to complete
run.wait_for_completion(show_output = True)